#### zigbang 매물 데이터 저장
- pip install geohash2

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import zigbang as zb
import pymongo
import pandas as pd

In [3]:
# server 연결
server = pymongo.MongoClient('mongodb://user:passwd@ip:27017/')
db = server.zigbang

In [15]:
addrs = {
    "seongnam": "성남동",
    "dangsan": "당산동",
    "hapjung": "합정동",
    "mongwon": "망원동",
    "sujin": "수진동",
    "yeungdeungpo":"영등포동"
}

In [16]:
# 데이터 수집 후 저장
for collection_name, addr in addrs.items():
    collection = db[collection_name]
    datas = zb.oneroom(addr)
    ids = collection.insert(datas)
    print(collection_name, addr, len(ids))

seongnam 성남동 92
dangsan 당산동 132
hapjung 합정동 142
mongwon 망원동 125
sujin 수진동 36
yeungdeungpo 영등포동 143


In [17]:
# 성남동에서 월세 50이상 보증금 5000에서 10000조건으로 검색
QUERY = {"rent": {"$lte": 50}, "deposit": {"$lte": 10000, "$gte": 5000}}
results = db["sujin"].find(QUERY)
df = pd.DataFrame(results).tail()
columns = ["title", "service_type", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor", "address1", "manage_cost", "is_new"]
df[columns]

,title,service_type,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost,is_new
22,대출가능 깔끔한 투룸7000( 2000-30)만원,빌라,전세,7000,0,66.1200,1,2,경기도 성남시 수정구 수진동,0,False
23,수진성당 뒷편 전세6000만원~,빌라,전세,6000,0,49.5900,2,2,경기도 성남시 수정구 수진동,0,False
24,"수진1구역,수진역10분거리,태평역12분거리,올수리",빌라,전세,10000,0,38.5000,1,1,경기도 성남시 수정구 수진동,0,True
25,수진1동 평지 중앙시장 가까움 투룸7500만원~,빌라,전세,7500,0,69.4200,1,2,경기도 성남시 수정구 수진동,0,False
26,준신축 주차가 보6000만원~~,빌라,전세,6000,0,36.3636,2,2,경기도 성남시 수정구 수진동,0,False


In [15]:
# 컬렉션 삭제
for addr in addrs:
    print(addr)
    server.zigbang.drop_collection(addr)

mongwon
hapjung


In [16]:
# 데이터 베이스 삭제
server.drop_database("zigbang")